Load packages

In [ ]:
#loading packages
push!(LOAD_PATH, "src");
import Base: @kwdef
using Test
using LifeInsuranceDataModel

using BitemporalPostgres
using SearchLight
using SearchLightPostgreSQL
using TimeZones
using ToStruct
using JSON

In [ ]:
purging the data model entirely - empty the schema

In [ ]:
if (haskey(ENV, "GENIE_ENV") && ENV["GENIE_ENV"] == "dev")
    if (haskey(ENV, "GITPOD_REPO_ROOT"))
        run(```psql -f sqlsnippets/droptables.sql```)
    else
        println("execute this in Your REPL")run(```sudo -u postgres psql -f sqlsnippets/droptables.sql```)
        println("""""")
    end
end

Loading the data model- Create tables, constraints etc. and load Roles

loading inverses of the role tables to provide role descriptions in object creation,  for instance like in: "ref_role=cpRole["Policy Holder"]

In [ ]:
LifeInsuranceDataModel.load_model()

cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end


Or just connect to an existing model

In [ ]:
LifeInsuranceDataModel.connect()

Create a Partner

In [ ]:
# create Partner
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1")
w = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)

In [ ]:
find(Partner)

Create tariffs

In [ ]:
# create Tariffs
function create_tariff(dsc)
    t = LifeInsuranceDataModel.Tariff()
    tr = LifeInsuranceDataModel.TariffRevision(description=dsc)
    w = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    create_entity!(w)
    create_component!(t, tr, w)
    commit_workflow!(w)
    t.id
end


LifeRiskTariff=create_tariff("Life Risk Insurance")
TerminalIllnessTariff=create_tariff("Terminal Illness")
OccupationalDisability=create_tariff("Occupational Disability")
ProfitParticipationTariff=create_tariff("Profit participation")

find(TariffRevision)

In [ ]:
find(Tariff,SQLWhereExpression("id=?",ProfitParticipationTariff))

In [ ]:


map(find(Tariff)) do t
    SearchLight.delete(t)
end

find(Tariff)

